In [15]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import dash
import pandas as pd
import polars as pl
import datetime as dt
import plotly.express as px
from google.cloud import storage
from configparser import ConfigParser
from google.oauth2 import service_account
from googleapiclient.discovery import build
from utils.demo_io import (
    get_initial_slide_df,
    get_fovs_df,
    get_top_level_dirs,
    populate_slide_rows,
    get_histogram_df,
    get_image,
)
import polars as pl
from gcsfs import GCSFileSystem
from PIL import Image
from io import BytesIO

# Parse in key and bucket name from config file
cfp = ConfigParser()
cfp.read("config.ini")

service_account_key_json = cfp["GCS"]["gcs_storage_key"]
gs_url = cfp["GCS"]["bucket_url"]

bucket_name = gs_url.replace("gs://", "")

# Define GCS file system so files can be read
gcs = GCSFileSystem(token=service_account_key_json)

# Authenticate using the service account key file
credentials = service_account.Credentials.from_service_account_file(
    service_account_key_json, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Create a storage client
storage_service = build("storage", "v1", credentials=credentials)


In [10]:
page_name = "595-AS13_2022-04-13_12-49-37.776201"

In [11]:
fovs_df = get_fovs_df(storage_service, bucket_name, [page_name])

In [22]:
def get_image(
    storage_service,
    bucket_name,
    slide_name,
    uri,
    resize_factor=1.0,
):
    """
    :brief: returns a file object corresponding to the image at the given uri
    :param uri: uri of the image, omitting bucket name
    """
    prefix = slide_name
    if not prefix.endswith("/"):
        prefix += "/"
    prefix += "spot_detection_result/"
    image = Image.open(
        BytesIO(
            (
                storage_service.objects()
                .get_media(bucket=bucket_name, object=(prefix + uri))
                .execute()
            )
        )
    )
    image = image.resize((int(image.size[0] * resize_factor), int(image.size[1] * resize_factor)))
    return image

In [23]:
get_image(
    storage_service,
    bucket_name,
    page_name,
    fovs_df["image_uri"][0].split("/")[-1],
    resize_factor=0.1
).show()